#### Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests
from bs4 import BeautifulSoup

import numpy as np # library to handle data in a vectorized manner

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


To transform the data into a pandas dataframe

In [2]:
page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
bs = BeautifulSoup(page,'xml')

In [3]:
table = bs.find('table',{'class':'wikitable sortable'})
table_rows = table.find_all('tr')

In [4]:
data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

In [5]:
df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df = df[(~df['PostalCode'].isnull()) & (df['Borough'] != 'Not assigned')]
df = df.groupby(['PostalCode', 'Borough'])['Neighbourhood'].apply(list) 
df = pd.DataFrame(df)
df1 = df.sort_values(by=['Neighbourhood'])
df1 = df1.reset_index()
df1.head(10)

,PostalCode,Borough,Neighbourhood
0,M5H,Downtown Toronto,"[Adelaide, King, Richmond]"
1,M1S,Scarborough,[Agincourt]
2,M1V,Scarborough,"[Agincourt North, L'Amoreaux East, Milliken, S..."
3,M9V,Etobicoke,"[Albion Gardens, Beaumond Heights, Humbergate,..."
4,M8W,Etobicoke,"[Alderwood, Long Branch]"
5,M3H,North York,"[Bathurst Manor, Downsview North, Wilson Heights]"
6,M2K,North York,[Bayview Village]
7,M5M,North York,"[Bedford Park, Lawrence Manor East]"
8,M5E,Downtown Toronto,[Berczy Park]
9,M1N,Scarborough,"[Birch Cliff, Cliffside West]"


In [7]:
#df3 = df1.loc[:,'Neighbourhood'].tolist()
df4 = ','.join(df1)
#df1['Neighbourhood'] = df1['Neighbourhood'].str.get()
df4

'PostalCode,Borough,Neighbourhood'

#### In this notebook, I used Beatifullsoup library to scrap the Wikipedia page. Then, I transforme the data in format requiered, cleaning data remove the Borough whit Not assigned and missings

In [10]:
df1.shape

(103, 3)

In [11]:
path ='http://cocl.us/Geospatial_data'
dfl = pd.read_csv(path)
dfl.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [13]:
dfc = pd.merge(df1, dfl, left_on ='PostalCode', right_on = 'Postal Code')
dfc.head()

,PostalCode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M5H,Downtown Toronto,"[Adelaide, King, Richmond]",M5H,43.650571,-79.384568
1,M1S,Scarborough,[Agincourt],M1S,43.794200,-79.262029
2,M1V,Scarborough,"[Agincourt North, L'Amoreaux East, Milliken, S...",M1V,43.815252,-79.284577
3,M9V,Etobicoke,"[Albion Gardens, Beaumond Heights, Humbergate,...",M9V,43.739416,-79.588437
4,M8W,Etobicoke,"[Alderwood, Long Branch]",M8W,43.602414,-79.543484


In [14]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(dfc['Borough'].unique()),
        dfc.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


In [15]:
address = ' Toronto, Ontario'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


Create map of Toront using latitude and longitude values

In [16]:
# create map of Toront using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neightbourhood in zip(dfc['Latitude'], dfc['Longitude'], dfc['Borough'], dfc['Neighbourhood']):
    label = '{}, {}'.format(neightbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [17]:
CLIENT_ID = 'ZVQR4LO4HEVNDYY5IG23KBDF2OOA00HWLLOM5FFTX5YHOQRB' # your Foursquare ID
CLIENT_SECRET = 'QR5M3BNEU5UQZGOXYKNEBPZLMPA3O3HC54ISAERZOKAOWCCA' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZVQR4LO4HEVNDYY5IG23KBDF2OOA00HWLLOM5FFTX5YHOQRB
CLIENT_SECRET:QR5M3BNEU5UQZGOXYKNEBPZLMPA3O3HC54ISAERZOKAOWCCA


In [18]:
dfc.loc[0, 'Neighbourhood']

['Adelaide', 'King', 'Richmond']

In [19]:
neighborhood_latitude = dfc.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = dfc.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = dfc.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of ['Adelaide', 'King', 'Richmond'] are 43.65057120000001, -79.3845675.


Let's create the GET request URL. Name your URL url.

In [20]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=ZVQR4LO4HEVNDYY5IG23KBDF2OOA00HWLLOM5FFTX5YHOQRB&client_secret=QR5M3BNEU5UQZGOXYKNEBPZLMPA3O3HC54ISAERZOKAOWCCA&v=20180605&ll=43.65057120000001,-79.3845675&radius=500&limit=100'

In [21]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d235dafd69ed000396f5e6d'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Financial District',
  'headerFullLocation': 'Financial District, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 131,
  'suggestedBounds': {'ne': {'lat': 43.65507120450001,
    'lng': -79.37835988143604},
   'sw': {'lat': 43.64607119550001, 'lng': -79.39077511856397}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ad4c062f964a520e5f720e3',
       'name': 'Four Seasons Centre for the Performing Arts',
       'location': {'address': '145 Queen St. W',
        'crossStreet': 'at University Ave.',
        'lat': 43.650592,
        'lng': -79.38

Let's create a function to repeat the same process to all the neighborhoods in Manhattan

In [22]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [23]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Four Seasons Centre for the Performing Arts,Concert Hall,43.650592,-79.385806
1,The Keg Steakhouse & Bar,Steakhouse,43.649937,-79.384196
2,Nathan Phillips Square,Plaza,43.652270,-79.383516
3,Rosalinda,Vegetarian / Vegan Restaurant,43.650252,-79.385156
4,Shangri-La Toronto,Hotel,43.649129,-79.386557


In [24]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [26]:
toronto_venues = getNearbyVenues(names = dfc['Neighbourhood'],
                                   latitudes = dfc['Latitude'],
                                   longitudes = dfc['Longitude']
                                  )


['Adelaide', 'King', 'Richmond']
['Agincourt']
['Agincourt North', "L'Amoreaux East", 'Milliken', 'Steeles East']
['Albion Gardens', 'Beaumond Heights', 'Humbergate', 'Jamestown', 'Mount Olive', 'Silverstone', 'South Steeles', 'Thistletown']
['Alderwood', 'Long Branch']
['Bathurst Manor', 'Downsview North', 'Wilson Heights']
['Bayview Village']
['Bedford Park', 'Lawrence Manor East']
['Berczy Park']
['Birch Cliff', 'Cliffside West']
['Bloordale Gardens', 'Eringate', 'Markland Wood', 'Old Burnhamthorpe']
['Brockton', 'Exhibition Place', 'Parkdale Village']
['Business Reply Mail Processing Centre 969 Eastern']
['CFB Toronto', 'Downsview East']
['CN Tower', 'Bathurst Quay', 'Island airport', 'Harbourfront West', 'King and Spadina', 'Railway Lands', 'South Niagara']
['Cabbagetown', 'St. James Town']
['Caledonia-Fairbanks']
['Canada Post Gateway Processing Centre']
['Cedarbrae']
['Central Bay Street']
['Chinatown', 'Grange Park', 'Kensington Market']
['Christie']
['Church and Wellesley']
['

In [27]:
print(toronto_venues.shape)
toronto_venues.tail()

(2258, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
2253,[York Mills West],43.752758,-79.400049,TD Canada Trust Branch and ATM,43.750923,-79.401004,Bank
2254,[York Mills West],43.752758,-79.400049,Kitchen Food Fair,43.751298,-79.401393,Convenience Store
2255,[York Mills West],43.752758,-79.400049,Tournament Park,43.751257,-79.399717,Park
2256,[York Mills West],43.752758,-79.400049,Yorkminster Park,43.755489,-79.401712,Park
2257,[York Mills West],43.752758,-79.400049,Jays Pub,43.752142,-79.405016,Bar


In [28]:
column_names = ['vecis'] 
neights = pd.DataFrame(columns=column_names)
neights

,vecis


In [29]:
prueba = toronto_venues.loc[:,'Neighborhood']

for row in prueba:
    prueba1 = ','.join(row)
    neights = neights.append({'vecis' : prueba1}, ignore_index = True)

neights.head()

,vecis
0,"Adelaide,King,Richmond"
1,"Adelaide,King,Richmond"
2,"Adelaide,King,Richmond"
3,"Adelaide,King,Richmond"
4,"Adelaide,King,Richmond"


In [31]:
torn_venues = toronto_venues.drop(['Neighborhood'], axis= 1)
trt_venues = pd.merge(torn_venues, neights, left_index=True, right_index=True)
trt_venues.head()


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,vecis
0,43.650571,-79.384568,Four Seasons Centre for the Performing Arts,43.650592,-79.385806,Concert Hall,"Adelaide,King,Richmond"
1,43.650571,-79.384568,The Keg Steakhouse & Bar,43.649937,-79.384196,Steakhouse,"Adelaide,King,Richmond"
2,43.650571,-79.384568,Nathan Phillips Square,43.652270,-79.383516,Plaza,"Adelaide,King,Richmond"
3,43.650571,-79.384568,Rosalinda,43.650252,-79.385156,Vegetarian / Vegan Restaurant,"Adelaide,King,Richmond"
4,43.650571,-79.384568,Shangri-La Toronto,43.649129,-79.386557,Hotel,"Adelaide,King,Richmond"


In [32]:
trt_venues.groupby('vecis').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
vecis,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",3,3,3,3,3,3
"Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown",11,11,11,11,11,11
"Alderwood,Long Branch",10,10,10,10,10,10
"Bathurst Manor,Downsview North,Wilson Heights",18,18,18,18,18,18
Bayview Village,4,4,4,4,4,4
"Bedford Park,Lawrence Manor East",25,25,25,25,25,25
Berczy Park,55,55,55,55,55,55


In [33]:
print('There are {} uniques categories.'.format(len(trt_venues['Venue Category'].unique())))

There are 280 uniques categories.


In [34]:
# one hot encoding
toronto_onehot = pd.get_dummies(trt_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['vecis'] = trt_venues['vecis'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,vecis,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Wo

In [35]:
toronto_onehot.shape

(2258, 281)

Let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category¶

In [36]:
toronto_grouped = toronto_onehot.groupby('vecis').mean().reset_index()
toronto_grouped

,vecis,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Wo

In [37]:
toronto_grouped.shape

(100, 281)

In [38]:
num_top_venues = 5

for hood in toronto_grouped['vecis']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['vecis'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
             venue  freq
0      Coffee Shop  0.06
1             Café  0.05
2              Bar  0.04
3       Steakhouse  0.04
4  Thai Restaurant  0.04


----Agincourt----
                venue  freq
0      Sandwich Place  0.25
1      Breakfast Spot  0.25
2              Lounge  0.25
3  Chinese Restaurant  0.25
4   Accessories Store  0.00


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
                venue  freq
0          Playground  0.33
1    Asian Restaurant  0.33
2                Park  0.33
3   Accessories Store  0.00
4  Miscellaneous Shop  0.00


----Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown----
                 venue  freq
0        Grocery Store  0.18
1             Pharmacy  0.09
2           Beer Store  0.09
3         Liquor Store  0.09
4  Fried Chicken Joint  0.09


----Alderwood,Long Branch----
            venue  freq
0     Pizza Place   0.2
1        Pharmacy   0.1
2  Sandw

                        venue  freq
0                      Bakery   0.2
1  Construction & Landscaping   0.2
2                        Park   0.2
3               Deli / Bodega   0.2
4            Basketball Court   0.2


----East Birchmount Park,Ionview,Kennedy Park----
                        venue  freq
0              Discount Store  0.25
1                  Playground  0.25
2            Department Store  0.25
3                 Coffee Shop  0.25
4  Modern European Restaurant  0.00


----East Toronto----
                venue  freq
0   Convenience Store  0.33
1         Coffee Shop  0.33
2                Park  0.33
3   Accessories Store  0.00
4  Miscellaneous Shop  0.00


----Emery,Humberlea----
                 venue  freq
0       Baseball Field   1.0
1    Accessories Store   0.0
2   Miscellaneous Shop   0.0
3                Motel   0.0
4  Monument / Landmark   0.0


----Fairview,Henry Farm,Oriole----
                  venue  freq
0        Clothing Store  0.12
1  Fast Food Restaurant  0.0

                venue  freq
0                Café  0.06
1         Coffee Shop  0.06
2               Hotel  0.05
3          Restaurant  0.05
4  Italian Restaurant  0.03


----Stn A PO Boxes 25 The Esplanade----
                venue  freq
0         Coffee Shop  0.11
1          Restaurant  0.04
2                Café  0.04
3  Italian Restaurant  0.03
4  Seafood Restaurant  0.03


----Studio District----
                 venue  freq
0                 Café  0.11
1          Coffee Shop  0.08
2               Bakery  0.05
3            Gastropub  0.05
4  American Restaurant  0.05


----The Annex,North Midtown,Yorkville----
               venue  freq
0        Coffee Shop  0.13
1     Sandwich Place  0.13
2               Café  0.13
3        Pizza Place  0.09
4  Indian Restaurant  0.04


----The Beaches----
                  venue  freq
0     Health Food Store   0.2
1                 Trail   0.2
2          Neighborhood   0.2
3  Other Great Outdoors   0.2
4                   Pub   0.2


----The Beac

In [39]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [40]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['vecis']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhood_venues_sorted = pd.DataFrame(columns=columns)
neighborhood_venues_sorted['vecis'] = toronto_grouped['vecis']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhood_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhood_venues_sorted.head()

,vecis,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Thai Restaurant,Bar,American Restaurant,Steakhouse,Gym,Cosmetics Shop,Breakfast Spot,Hotel
1,Agincourt,Lounge,Breakfast Spot,Sandwich Place,Chinese Restaurant,Yoga Studio,Drugstore,Discount Store,Dog Run,Doner Restaurant,Donut Shop
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Park,Playground,Asian Restaurant,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Fried Chicken Joint,Pizza Place,Sandwich Place,Fast Food Restaurant,Liquor Store,Coffee Shop,Beer Store,Video Store,Pharmacy
4,"Alderwood,Long Branch",Pizza Place,Gym,Coffee Shop,Pharmacy,Skating Rink,Pub,Dance Studio,Pool,Sandwich Place,Diner


In [41]:
column_names = ['vecis'] 
neights1 = pd.DataFrame(columns=column_names)
neights1

,vecis


In [42]:
prueba1 = dfc.loc[:,'Neighbourhood']

for row in prueba1:
    prueba11 = ','.join(row)
    neights1 = neights1.append({'vecis' : prueba11}, ignore_index = True)

neights1.head()

,vecis
0,"Adelaide,King,Richmond"
1,Agincourt
2,"Agincourt North,L'Amoreaux East,Milliken,Steel..."
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."
4,"Alderwood,Long Branch"


In [43]:
torn_data = dfc.drop(['Neighbourhood'], axis= 1)
toronto_data = pd.merge(torn_data, neights1, left_index=True, right_index=True)
toronto_data.head()

,PostalCode,Borough,Postal Code,Latitude,Longitude,vecis
0,M5H,Downtown Toronto,M5H,43.650571,-79.384568,"Adelaide,King,Richmond"
1,M1S,Scarborough,M1S,43.794200,-79.262029,Agincourt
2,M1V,Scarborough,M1V,43.815252,-79.284577,"Agincourt North,L'Amoreaux East,Milliken,Steel..."
3,M9V,Etobicoke,M9V,43.739416,-79.588437,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."
4,M8W,Etobicoke,M8W,43.602414,-79.543484,"Alderwood,Long Branch"


Run k-means to cluster the neighborhood into 5 clusters.

In [90]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('vecis', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=5).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 4, 0, 0, 0, 0, 0, 0, 0])

In [93]:
# add clustering labels
#neighborhood_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhood_venues_sorted.set_index('vecis'), on='vecis')

#toronto_merged['Clusteres'] = toronto_merged['Cluster Labels'].astype('int')
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].fillna(2).astype(int)

toronto_merged.head() # check the last columns!


,PostalCode,Borough,Postal Code,Latitude,Longitude,vecis,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5H,Downtown Toronto,M5H,43.650571,-79.384568,"Adelaide,King,Richmond",0,Coffee Shop,Café,Thai Restaurant,Bar,American Restaurant,Steakhouse,Gym,Cosmetics Shop,Breakfast Spot,Hotel
1,M1S,Scarborough,M1S,43.794200,-79.262029,Agincourt,0,Lounge,Breakfast Spot,Sandwich Place,Chinese Restaurant,Yoga Studio,Drugstore,Discount Store,Dog Run,Doner Restaurant,Donut Shop
2,M1V,Scarborough,M1V,43.815252,-79.284577,"Agincourt North,L'Amoreaux East,Milliken,Steel...",1,Park,Playground,Asian Restaurant,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
3,M9V,Etobicoke,M9V,43.739416,-79.588437,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",0,Grocery Store,Fried Chicken Joint,Pizza Place,Sandwich Place,Fast Food Restaurant,Liquor Store,Coffee Shop,Beer Store,Video Store,Pharmacy
4,M8W,Etobicoke,M8W,43.602414,-79.543484,"Alderwood,Long Branch",0,Pizza Place,Gym,Coffee Shop,Pharmacy,Skating Rink,Pub,Dance Studio,Pool,Sandwich Place,Diner


Let's visualize the resulting clusters

In [94]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['vecis'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters